In [1]:
from DemoServerDB import ConnectDatabase

In [2]:
temp = ConnectDatabase()

In [3]:
query = 'SELECT * from master_table_hist'

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

In [5]:
temp.cur.execute(query)

In [6]:
col_names = [desc[0] for desc in temp.cur.description]

In [7]:
master_table_his = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [8]:
master_table_his.head()

,id,cust_id,churn,engagement,percent_churn,cltv,value,percent_cltv,high_convertor,bio,lifestage,custom_date_identifier
0,4764,6151721,None,None,None,None,None,None,High,0,None,2017-04-01
1,5116,6230240,None,None,None,None,None,None,Low,0,None,2017-04-01
2,377,6310287,None,None,None,None,None,None,High,9,None,2017-04-01
3,5595,6167409,0,Very Low,0,0,Very Low,0,None,None,-,2017-04-01
4,3805,6266404,0,Low,0.13447828904731,0,Very Low,0,High,4,Lapsed,2017-04-01


In [9]:
master_table_his.tail()

,id,cust_id,churn,engagement,percent_churn,cltv,value,percent_cltv,high_convertor,bio,lifestage,custom_date_identifier
14632,3288,6307811,1,Low,0.13447828904731,40,Low,0,High,7,Lapsed,2017-04-03
14633,3290,6333298,1,Low,None,40,Low,None,Low,0,None,2017-04-03
14634,3289,6202240,1,Low,0.967919637070642,40,Low,75.0162022034997,Low,7,Retained,2017-04-03
14635,3291,6339049,1,Low,None,40,Low,None,Low,0,None,2017-04-03
14636,3292,6190868,1,Low,None,40,Low,None,Low,0,None,2017-04-03


In [10]:
master_table_his.shape

(14637, 12)

In [ ]:
len(master_table_his.cust_id.unique())

In [11]:
master_table_his.custom_date_identifier.unique()

array(['2017-04-01T00:00:00.000000000', '2017-04-02T00:00:00.000000000',
       '2017-04-03T00:00:00.000000000'], dtype='datetime64[ns]')

In [12]:
master_table_his.churn.unique()

array([None, Decimal('0'), Decimal('1')], dtype=object)

In [13]:
master_table_his.value.unique()

array([None, 'Very Low', 'Low', 'Medium', 'High', 'Very High'], dtype=object)

In [14]:
master_table_his.engagement.unique()

array([None, 'Very Low', 'Low', 'Medium', 'High', 'Very High'], dtype=object)

In [ ]:
list_of_indexes_new = [np.argmax(g['custom_date_identifier']) for l, g in master_table_his.groupby(['cust_id'])]

In [ ]:
new_data = master_table_his.ix[list_of_indexes_new]

In [ ]:
new_data.reset_index(drop=True, inplace=True)

In [ ]:
new_data.head()

In [ ]:
new_data.tail()

In [ ]:
new_data.custom_date_identifier.unique()

In [ ]:
new_data.shape

In [ ]:
master_table_his.shape

In [ ]:
xyz = master_table_his.groupby(['cust_id', 'custom_date_identifier']).size().reset_index()

In [ ]:
def x(frame):
    if len(frame.custom_date_identifier) > 1:
        print frame.custom_date_identifier
        if frame.custom_date_identifier.iloc[-2] == '10':
            frame['time'] = frame.custom_date_identifier.iloc[-2]
        else:
            frame['time'] = frame.custom_date_identifier.iloc[-1]
    else:
        frame['time'] = frame.custom_date_identifier.iloc[-1]
    return frame

In [ ]:
dates = xyz.groupby('cust_id').apply(x)

In [ ]:
del dates['custom_date_identifier']

In [ ]:
dates = dates.drop_duplicates('cust_id')

In [ ]:
dates.head()

In [ ]:
dates.rename(columns={'time':'custom_date_identifier'}, inplace=True)

In [ ]:
dates.head()

In [ ]:
master_table_his.head()

In [ ]:
old_data = pd.merge(master_table_his, dates, on=['cust_id', 'custom_date_identifier'], how='left')

In [ ]:
old_data.head()

In [ ]:
old_data.custom_date_identifier.unique()

In [ ]:
old_data.shape

In [ ]:
master_table_his.shape

In [ ]:
old_data.custom_date_identifier.describe()

In [ ]:
old_data.custom_date_identifier.unique()

In [ ]:
cltv_value_old = old_data[['cltv', 'custom_date_identifier']]

In [ ]:
old_cltv_sum = cltv_value_old.groupby('custom_date_identifier').sum().reset_index()

In [ ]:
old_cltv_count = cltv_value_old.groupby('custom_date_identifier').count().reset_index()

In [ ]:
old_cltv_avrg = old_cltv_sum.merge(old_cltv_count, on='custom_date_identifier')

In [ ]:
old_cltv_avrg.cltv_x = old_cltv_avrg.cltv_x.astype('float64', raise_on_error = False)
old_cltv_avrg.cltv_y = old_cltv_avrg.cltv_y.astype('float64', raise_on_error = False)

In [ ]:
old_cltv_avrg['average_cltv'] = old_cltv_avrg.cltv_x / old_cltv_avrg.cltv_y.replace({ 0 : np.nan })

In [ ]:
old_cltv_avrg

In [ ]:
cltv_value_new = new_data[['cltv', 'custom_date_identifier']]

In [ ]:
new_cltv_sum = cltv_value_new.groupby('custom_date_identifier').sum().reset_index()

In [ ]:
new_cltv_count = cltv_value_new.groupby('custom_date_identifier').count().reset_index()

In [ ]:
new_cltv_avrg = new_cltv_sum.merge(new_cltv_count, on='custom_date_identifier')

In [ ]:
new_cltv_avrg.cltv_x = new_cltv_avrg.cltv_x.astype('float64', raise_on_error = False)
new_cltv_avrg.cltv_y = new_cltv_avrg.cltv_y.astype('float64', raise_on_error = False)

In [ ]:
new_cltv_avrg['average_cltv'] = new_cltv_avrg.cltv_x / new_cltv_avrg.cltv_y.replace({ 0 : np.nan })

In [ ]:
new_cltv_avrg